# Dimension Modeling

## Dim_model Sink - Initial and Incremental run

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType, LongType
from pyspark.sql.window import Window

# Get source data
df_src = spark.sql('''
    SELECT DISTINCT             subcategory_key,
                                subcategory_name
    FROM parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`
''')

# Initialize target DataFrame
if spark.catalog.tableExists('contoso_catalog.gold.dim_sub'):
    df_tgt = spark.sql('''
    SELECT                      subcategory_id, 
                                subcategory_key,
                                subcategory_name
        FROM contoso_catalog.gold.dim_sub
    ''')
else:
    df_tgt = df_src.withColumn("subcategory_id", F.lit(0).cast(LongType()))\
                   .withColumn("is_current", F.lit(1).cast(IntegerType()))\
                   .withColumn("start_date", F.current_date())\
                   .withColumn("end_date", F.lit(None).cast(DateType()))\
                   .filter("1 = 0")  # Empty Schema

def generate_surrogate_key(df, start_value):
    w = Window.orderBy("subcategory_key")
    return df.withColumn("subcategory_id", F.row_number().over(w) + F.lit(start_value))

def apply_scd_type2_changes(df_src, df_tgt):
    if df_tgt.rdd.isEmpty():
        return generate_surrogate_key(df_src, 0)\
               .withColumn("start_date", F.current_date())\
               .withColumn("end_date", F.lit(None).cast(DateType()))\
               .withColumn("is_current", F.lit(1).cast(IntegerType()))

    max_surrogate_key = df_tgt.agg(F.max("subcategory_id")).collect()[0][0]
    
    src = df_src.alias("src")
    tgt = df_tgt.filter(F.col("is_current") == 1).alias("tgt")
    
    joined_df = src.join(tgt, "subcategory_key", "outer")
    
    new_records = joined_df.filter(F.col("tgt.subcategory_id").isNull())
    new_records_count = new_records.count()
    
    if new_records.count() > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))

    changed_records = joined_df.filter((F.col("tgt.subcategory_id").isNotNull()) &
                                       
                                        (F.coalesce(src.subcategory_name != tgt.subcategory_name, F.lit(False)))
                                        
                                        )   
    
    changed_records_count = changed_records.count()
    
    if new_records_count == 0 and changed_records_count == 0:
        return df_tgt
    
    if new_records_count > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))
    
    if changed_records_count > 0:
        new_versions = changed_records.select(
            "src.*"
        ).drop("subcategory_id", "start_date", "end_date", "is_current")
        
        start_key = max_surrogate_key + new_records_count + 1
        new_versions = generate_surrogate_key(new_versions, start_key)\
                      .withColumn("start_date", F.current_date())\
                      .withColumn("end_date", F.lit(None).cast(DateType()))\
                      .withColumn("is_current", F.lit(1).cast(IntegerType()))

        old_versions = df_tgt.join(changed_records.select("subcategory_key"), "subcategory_key", "inner")\
                            .withColumn("end_date", F.when(F.col("is_current") == 1, F.date_sub(F.current_date(), 1)).otherwise(F.col("end_date")))\
                            .withColumn("is_current", F.when(F.col("is_current") == 1, F.lit(0)).otherwise(F.col("is_current")))

        unchanged_records = df_tgt.join(changed_records.select("subcategory_key"), "subcategory_key", "leftanti")
        
        # Build the final DataFrame based on what we have
        final_df = unchanged_records
        
        if changed_records_count > 0:
            final_df = final_df.unionAll(old_versions).unionAll(new_versions)
            
        if new_records_count > 0:
            final_df = final_df.unionAll(new_records)
            
        return final_df

# Apply SCD Type 2 changes
result_df = apply_scd_type2_changes(df_src, df_tgt)

display(result_df)

subcategory_id,subcategory_key,subcategory_name
1,101,MP4&MP3
2,104,Recording Pen
3,106,Bluetooth Headphones
4,201,Televisions
5,202,VCD & DVD
6,203,Home Theater System
7,205,Car Video
8,301,Laptops
9,303,Desktops
10,304,Monitors


## Write data to catalog

In [0]:
result_df.write.format('delta')\
                .mode('overwrite')\
                    .option('path', 'abfss://gold@contosoprojectstorage.dfs.core.windows.net/dim_sub')\
                        .saveAsTable('contoso_catalog.gold.dim_sub')